In [39]:
import json
import matplotlib.pyplot as plt
import numpy as np
import operator
import pandas as pd
import re
import seaborn as sns

In [40]:
with open('../data/flutter/flutter_issues_labeled.json') as json_data:
    issues = json.load(json_data)
print("Number of issues: " + str(len(issues)))
labeled_issues = [issue for issue in issues if len(issue['completed_by']) > 0]
print("Number of labeled issues: " + str(len(labeled_issues)))

Number of issues: 7170
Number of labeled issues: 2504


In [41]:
# create a data frame from the list of issues
df_list = []
for issue in labeled_issues[25:]:
        df_dict = {}
        df_dict['comments'] = issue['comments']
        if (not issue['body']):
            issue['body'] = ""
        df_dict['title'] = issue['title']
        df_dict['body'] = issue['body']
        df_dict['closed_date'] = pd.to_datetime(issue['closed_at'])
        df_dict['created_date'] = pd.to_datetime(issue['created_at'])
        df_dict['completed_by'] = issue['completed_by']
        df_dict['labels'] = [label['name'] for label in issue['labels']]
        df_dict['assignees'] = [assignee['login'] for assignee in issue['assignees']]
        df_list.append(df_dict)
df = pd.DataFrame(df_list).sort_values('closed_date')
df.head(2)
print(len(df))

2479


In [42]:
num_correct_k = 0
num_correct_total = 0
num_total_k = 0
num_total = 0
k = 1
dev_counts = {}
devs = set()
for _, row in df.iterrows():
    for assignee in row['assignees']:
        if dev_counts.get(assignee, 0) < k:
            display(row)
            if assignee in row['completed_by']:
                num_correct_k += 1
            num_total_k += 1
        if assignee in row['completed_by']:
            num_correct_total += 1
        num_total += 1
    for completer in row['completed_by']:
        devs.add(completer)
        dev_counts[completer] = dev_counts.get(completer, 0) + 1
print("Num correct events for first k contributor: " + str(num_correct_k))
print("Total events for first k contributor: " + str(num_total_k))
print("Num correct events for all assignees: " + str(num_correct_total))
print("Total events for all assignees: " + str(num_total))
print("Total number of unique devs: " + str(len(devs)))

assignees                                                [abarth]
body            When I try to `flutter start` any of the examp...
closed_date                                   2015-11-09 20:13:32
comments                                                        3
completed_by                                     [abarth, DanTup]
created_date                                  2015-11-08 20:33:37
labels                                           [easy fix, tool]
title           `flutter start` doesn't give good error messag...
Name: 1, dtype: object

assignees                                         [collinjackson]
body                                              cc @tvolkert \n
closed_date                                   2015-11-10 00:40:38
comments                                                        1
completed_by                                      [collinjackson]
created_date                                  2015-11-09 21:07:13
labels                                                         []
title           Implement Dart's HTTP package in Flutter with ...
Name: 18, dtype: object

assignees                                         [HansMuller]
body                                         cc @HansMuller \n
closed_date                                2015-11-13 00:40:47
comments                                                     0
completed_by                                      [HansMuller]
created_date                               2015-11-12 20:27:39
labels                [framework, severe: regression, ⚠ TODAY]
title           closing the menu in stocks throws an exception
Name: 87, dtype: object

assignees                                          [chinmaygarde]
body            Currently `flutter ios --init` launches Xcode ...
closed_date                                   2016-01-26 20:05:52
comments                                                        3
completed_by                                       [chinmaygarde]
created_date                                  2016-01-25 22:12:14
labels                                    [tool, ⌺‬ platform-ios]
title                `flutter ios --init` should not launch XCode
Name: 291, dtype: object

assignees                                         [johnmccutchan]
body            Right now, breakpoints set, via Atom, on Flutt...
closed_date                                   2016-02-10 22:41:21
comments                                                        2
completed_by                                         [devoncarew]
created_date                                  2016-01-28 23:20:54
labels                                                     [tool]
title                     Breakpoints work starting at app launch
Name: 311, dtype: object

...

assignees                                           [sivachandra]
body            Hot Reload works fine.  Hot Restart crashes th...
closed_date                                   2017-07-12 23:44:44
comments                                                        8
completed_by                                        [sivachandra]
created_date                                  2017-04-15 05:30:57
labels                                      [t: hot reload, tool]
title           Hot Restart fails for apps using packages with...
Name: 1801, dtype: object

assignees                                               [sigurdm]
body            In the video example of the gallery, we see tw...
closed_date                                   2017-12-05 09:20:13
comments                                                        2
completed_by                                            [sigurdm]
created_date                                  2017-12-01 20:01:42
labels                                  [a: video, team: gallery]
title           Video example in gallery shows black square wh...
Name: 2209, dtype: object

assignees                                              [zanderso]
body            In #14610, we unified the Dart SDK against whi...
closed_date                                   2018-02-20 23:52:50
comments                                                        7
completed_by                                           [tvolkert]
created_date                                  2018-02-16 17:53:26
labels                                 [dependency: dart, engine]
title           Flutter's Dart SDK version breaks semantic ver...
Name: 2327, dtype: object

assignees                              [mattsarett]
body                                           \r\n
closed_date                     2018-04-24 23:43:11
comments                                         11
completed_by                           [mattsarett]
created_date                    2017-02-14 21:01:27
labels                                           []
title           Run Flutter with Color Correct Skia
Name: 1605, dtype: object

Num correct events for first k contributor: 16
Total events for first k contributor: 26
Num correct events for all assignees: 1415
Total events for all assignees: 1554
Total number of unique devs: 103
